# Packages

In [20]:
import geopandas as gpd
import os
import rasterio
import geopandas as gpd
import numpy as np
from rasterio.mask import mask
from rasterio.features import rasterize


## AOI
wd_shp = r'G:\Shared drives\Wellcome Trust Project Data\1_preprocess\UrbanCoolingModel'
aoi_shapefile = os.path.join(wd_shp, "London_Ward_aoi_prj.shp") 
# Load administrative boundaries
aoi = gpd.read_file(aoi_shapefile)


# LC input 


In [22]:
# --- File Paths ---
lc_path = r"G:\Shared drives\Wellcome Trust Project Data\1_preprocess\UrbanCoolingModel\EP_preliminary_tests\clipped_lulc\UKECH\LCM2023_London_10m.tif"

lc_code_new_tcc = 100  # Code for the new LC that overlaps with Tree Canopy Cover data


# Define labels
land_cover_labels = {
    1: 'Deciduous woodland',
    2: 'Coniferous woodland',
    3: 'Arable',
    4: 'Improved Grassland',
    5: 'Neutral Grassland',
    6: 'Calcareous Grassland',
    7: 'Acid grassland',
    8: 'Fen, Marsh, and Swamp',
    9: 'Heather',
    10: 'Heather grassland',
    11: 'Bog',
    12: 'Inland Rock',
    13: 'Saltwater',
    14: 'Freshwater',
    15: 'Supralittoral Rock',
    16: 'Supralittoral Sediment',
    17: 'Littoral Rock',
    18: 'Littoral Sediment',
    19: 'Saltmarsh',
    20: 'Urban',
    21: 'Suburban',
}


## clip to aoi

In [23]:

## clip raster to AOI -------------------------------------------------------------------

# load function to use in notebook
from function_clip_raster_to_aoi import clip_raster_to_aoi


## clip 
# Output path (optional)
lc_clipped_tif = lc_path.replace(".tif", "_clip2aoi.tif")

filled_arr, filled_transform, filled_profile = clip_raster_to_aoi(
    raster_path=lc_path,
    aoi=aoi_shapefile,  # or pass your GeoDataFrame
    out_path=lc_clipped_tif,
    replace_nodata_with=0,     # convert 255/nodata -> 0
    keep_nodata_tag=False      # keep False so 0 is treated as a real class
)


## Keep only the first band, as the UKECH land cover raster has two bands, but we only need the first one for land cover classification.
if filled_arr.ndim == 3:  # (bands, rows, cols)
    filled_arr = filled_arr[0, :, :]  # slice first band
    filled_profile.update(count=1)    # tell rasterio it's single-band now

# Save manually
with rasterio.open(lc_clipped_tif, "w", **filled_profile) as dst:
    dst.write(filled_arr, 1)


print("Done. Saved to:\n\t", lc_clipped_tif)
print(np.unique(filled_arr[~np.isnan(filled_arr)]))  # check valid classes





# --- Step 1: Load the Raster & apply mask ----------------------------------------------
with rasterio.open(lc_clipped_tif) as src:
    landcover_meta = src.meta.copy()
    landcover_crs  = src.crs
    transform      = src.transform
    arr            = src.read(1)
    src_nodata     = src.nodata

# build a combined mask: zeros and any existing src NoData
mask = (arr == 0)
if src_nodata is not None:
    mask |= (arr == src_nodata)



# coerce to plain ndarray using nodata=0
nodata = 0
landcover_meta.update(nodata=0)
landcover_data = np.where(mask, nodata, arr).astype(np.int32)


Done. Saved to:
	 G:\Shared drives\Wellcome Trust Project Data\1_preprocess\UrbanCoolingModel\EP_preliminary_tests\clipped_lulc\UKECH\LCM2023_London_10m_clip2aoi.tif
[ 0  1  2  3  4  5  6  8  9 10 12 13 14 16 18 19 20 21]


# TCC polygon

In [15]:

### Load opportunity land shapefile
shp_path = r"G:\Shared drives\Wellcome Trust Project Data\0_source_data\uk_shapefile_TCC24\TreeCanopyCover24_stitched.shp"

# --- Step 2: Load the Shapefile ---
polygon = gpd.read_file(shp_path)

## total area calculation
# Ensure CRS is projected (replace EPSG:XXXX with a suitable projection for your area, e.g., EPSG:5070 for US)
if polygon.crs.is_geographic:
    polygon = polygon.to_crs(epsg=5070)  # Albers Equal Area for US

# Calculate total area in square meters
total_area_m2 = polygon.geometry.area.sum()

# Optionally convert to square kilometers or hectares
total_area_km2 = total_area_m2 / 1e6

print(f"Total area of opportunity land cover: {total_area_km2:,.2f} km²")


Total area of opportunity land cover: 311.94 km²


# Reclassify LULC using TCC

In [24]:
lc_reclass = lc_clipped_tif.replace(".tif", "_tcc24.tif")


# --- Step 3: Reproject Shapefile if Needed ---
# (Skip if either CRS is None; otherwise align to raster CRS)
if polygon.crs and (polygon.crs != landcover_crs):
    polygon = polygon.to_crs(landcover_crs)

# Drop empty/invalid geometries (common source of rasterize errors)
polygon = polygon[polygon.geometry.notna() & ~polygon.geometry.is_empty]

# --- Step 4: Rasterize the Shapefile ---
shape_mask = rasterize(
    [(geom, 1) for geom in polygon.geometry],
    out_shape=landcover_data.shape,
    transform=landcover_meta["transform"],
    fill=0,
    dtype="uint8"
)


# --- Step 5: Apply the Mask to Update Land Cover Values ---

remapped2 = landcover_data.copy()

# Protect NoData if present
if nodata is not None:
    valid_mask = (landcover_data != nodata)
else:
    valid_mask = np.ones_like(landcover_data, dtype=bool)

target_mask = (shape_mask == 1) & valid_mask
remapped2[target_mask] = lc_code_new_tcc



# --- Step 6: Save the Updated Raster ---
landcover_meta.update(dtype=rasterio.uint8, compress="lzw")  # Ensure correct datatype

meta_out = landcover_meta.copy()
meta_out.update(
    dtype=rasterio.uint8,
    compress="lzw",
    nodata=nodata
)


with rasterio.open(lc_reclass, "w", **meta_out) as dst:
    dst.write(remapped2.astype(rasterio.uint8), 1)

print(f"Updated land cover raster saved at: {lc_reclass}")


# Scenario labels
land_cover_labels_scenario2 = land_cover_labels.copy()
# Add new LC label
land_cover_labels_scenario2[lc_code_new_tcc] = "Tree canopy"


Updated land cover raster saved at: G:\Shared drives\Wellcome Trust Project Data\1_preprocess\UrbanCoolingModel\EP_preliminary_tests\clipped_lulc\UKECH\LCM2023_London_10m_clip2aoi_tcc24.tif


### % of LC change - run

In [25]:

# load function to use in notebook
from function_summarize_lc_classes import summarize_lc_classes


# ---- Compute pixel area if projected (optional) ----
px_area_m2 = None
if landcover_crs and landcover_crs.is_projected:
    # rasterio Affine: transform.a = pixel width, transform.e = pixel height (negative)
    px_area_m2 = abs(transform.a) * abs(transform.e)

# ---- Summaries: BEFORE (original) and AFTER (remapped) ----
summary_before = summarize_lc_classes(
    landcover_data,
    land_cover_labels,
    nodata=nodata,
    px_area_m2=px_area_m2,
    sort_by="class"
)

# Optional: if you defined land_cover_labels_scenario; otherwise reuse land_cover_labels
labels_after = globals().get("land_cover_labels_scenario2", land_cover_labels)

summary_after = summarize_lc_classes(
    remapped2,
    labels_after,
    nodata=nodata,
    px_area_m2=px_area_m2,
    sort_by="class"
)

print("\n=== Class proportions BEFORE scenario ===")
print(summary_before.to_string(index=False))

print("\n=== Class proportions AFTER scenario ===")
print(summary_after.to_string(index=False))

# ---- Optional: quick change report for Tree->Built-up and Shrubland->Built-up ----
def class_count(df, code):
    row = df.loc[df["class_code"] == code, "count"]
    return int(row.iloc[0]) if len(row) else 0


# ---- Optional: save to CSV ----
out_csv_before = lc_reclass.replace(".tif", "_class_summary_BEFORE.csv")
out_csv_after  = lc_reclass.replace(".tif", "_class_summary_AFTER.csv")
summary_before.to_csv(out_csv_before, index=False)
summary_after.to_csv(out_csv_after, index=False)
print(f"\nSaved summaries:\n- {out_csv_before}\n- {out_csv_after}")


=== Class proportions BEFORE scenario ===
 class_code                  label   count  proportion  percent     area_m2  area_ha  area_km2
          1     Deciduous woodland  867944    0.054427    5.443  86794400.0  8679.44     86.79
          2    Coniferous woodland   65124    0.004084    0.408   6512400.0   651.24      6.51
          3                 Arable  597008    0.037437    3.744  59700800.0  5970.08     59.70
          4     Improved Grassland 3559567    0.223214   22.321 355956700.0 35595.67    355.96
          5      Neutral Grassland      11    0.000001    0.000      1100.0     0.11      0.00
          6   Calcareous Grassland   50726    0.003181    0.318   5072600.0   507.26      5.07
          8  Fen, Marsh, and Swamp   49809    0.003123    0.312   4980900.0   498.09      4.98
          9                Heather    1220    0.000077    0.008    122000.0    12.20      0.12
         10      Heather grassland    8112    0.000509    0.051    811200.0    81.12      0.81
       

### change summary

In [26]:
## compute the change in area_km2 ------------------------------------------------------------------ 
import pandas as pd

# Read
df_before = pd.read_csv(out_csv_before)
df_after  = pd.read_csv(out_csv_after)

# Keep only what we need and rename
b = df_before[['class_code', 'area_km2']].rename(columns={'area_km2': 'area_km2_before'})
a = df_after [['class_code', 'area_km2']].rename(columns={'area_km2': 'area_km2_after'})

# Outer join on class_code
merged = b.merge(a, on='class_code', how='outer')

# Ensure numeric, then compute change
for c in ['area_km2_before', 'area_km2_after']:
    merged[c] = pd.to_numeric(merged[c], errors='coerce')

merged['area_km2_change'] = merged['area_km2_after'].fillna(0) - merged['area_km2_before'].fillna(0)

# Save
out_csv_change = lc_reclass.replace(".tif", "_class_summary_CHANGE.csv")
merged.to_csv(out_csv_change, index=False)
print(f"Saved: {out_csv_change}")

print("\n=== Class proportions changed ===")
print(merged.to_string(index=False))

Saved: G:\Shared drives\Wellcome Trust Project Data\1_preprocess\UrbanCoolingModel\EP_preliminary_tests\clipped_lulc\UKECH\LCM2023_London_10m_clip2aoi_tcc24_class_summary_CHANGE.csv

=== Class proportions changed ===
 class_code  area_km2_before  area_km2_after  area_km2_change
          1            86.79            8.16           -78.63
          2             6.51            1.18            -5.33
          3            59.70           57.05            -2.65
          4           355.96          281.48           -74.48
          5             0.00            0.00             0.00
          6             5.07            4.14            -0.93
          8             4.98            3.15            -1.83
          9             0.12            0.09            -0.03
         10             0.81            0.48            -0.33
         12             1.29            1.29             0.00
         13             1.10            1.10             0.00
         14            33.59           